### First, setup the LLMs. Use Anthropic and OpenAI interchangeably.

In [1]:
from langchain.llms import OpenAI, Anthropic
from langchain import PromptTemplate, LLMChain

openai_llm = OpenAI(temperature=0.7)
anthropic_llm = Anthropic(temperature=0.7)

### Example 1a: Using PromptTemplate -- write a list of jokes about a given profession. Then write a second list of jokes similar to the first one but on a different profession

**Prompt, template etc. need to be defined only once**

In [2]:
professions = ['lawyers', 'truck drivers', 'writers', 'accountants', 'waitresses', 'pilots', 'programmers']
profession1 = professions[0]
prompt1 = f"Write 7 jokes about {profession1}. Return the jokes as a numbered list."

template1 = "Write 7 jokes about {profession1}. Return the jokes as a numbered list." # needed by the LLMchain demo

template = """
Given below is a numbered list of jokes about {profession1}. 
Riff on these jokes to make equal number of similar jokes about {profession2}. 
Do not add any explanation, elaboration or any additional comments.
---
{jokes_output}
"""

p_template = PromptTemplate(input_variables=['profession1', 'profession2', 'jokes_output'], template=template)

In [3]:
def print_jokes_multi(llm_in_use, prompt1, professions, p_template):
    resp1 = llm_in_use(prompt1)
    profession1 = professions[0]
    print (f"================== ORIGINAL jokes : {profession1} ========================")
    print (resp1.strip())

    for profession2 in professions[1:]:
        prompt2 = p_template.format(profession2=profession2, jokes_output=resp1, profession1=profession1)
        resp2 = llm_in_use(prompt2)
        print (f"======================={profession2}=====================")
        print (resp2.strip())

**First, run on Anthropic**

In [4]:
print_jokes_multi(anthropic_llm, prompt1, professions, p_template)

================== ORIGINAL jokes : lawyers ========================
1. Why do lawyers wear neckties? To keep the foreskin from crawling up their faces.

2. How can you tell when a lawyer is lying? His lips are moving.

3. What's the difference between a jellyfish and a lawyer? One's a form of life, the other is a form of slime.

4. What's the difference between a lawyer and a vulture? The lawyer has better skin and shorter feathers.

5. How many lawyers does it take to change a light bulb? Three. One to argue for hours that it's not necessary to change the bulb, one to concede that it is necessary and bill eight hours for research, and one to screw in the new bulb after the client has paid for it all.

6. Why did God make snakes just before lawyers? To practice.

7. What do you call 25 lawyers skydiving from an airplane? Skeet.
=======================truck drivers=====================
1. Why do truck drivers wear baseball caps? To keep their mullets from crawling up their faces.

2. H

**Next, OpenAI**

In [5]:
# only need to change the first argument, i.e. the LLM object
print_jokes_multi(openai_llm, prompt1, professions, p_template)

================== ORIGINAL jokes : lawyers ========================
1. What do you call a lawyer with an IQ of 50? Your Honor.
2. What did the lawyer name his daughter? Sue.
3. Why did the lawyer cross the road? He was suing the chicken on the other side.
4. What's the difference between a good lawyer and a bad lawyer? A bad lawyer can let a case drag on for several years. A good lawyer can make it last even longer.
5. What did the lawyer name his son? Billable.
6. Why don't lawyers go to the beach? Because cats keep trying to bury them in the sand.
7. How can you tell when a lawyer is lying? His lips are moving.
=======================truck drivers=====================
1. What do you call a truck driver with an IQ of 50? Overdrive.
2. What did the truck driver name his daughter? Mack.
3. Why did the truck driver cross the road? To get to the weigh station.
4. What's the difference between a good truck driver and a bad truck driver? A bad truck driver can take all day to get to his de

### Example 1b -- same objective as above, using the `predict()` function of `LLMchain` instead of calling `llm()`

In [6]:
# using LLMchain
def print_jokes_multi_llmchain(llm_in_use, template1, professions, p_template, verbose=False):
    prompt1_template = PromptTemplate(input_variables=['profession1'], template=template1)
    profession1 = professions[0]
    llm_chain1 = LLMChain(prompt=prompt1_template, llm=llm_in_use, verbose=verbose)
    resp1 = llm_chain1.predict(profession1=profession1)
        
    print (f"================== ORIGINAL jokes : {profession1} ========================")
    print (resp1.strip())

    for pff in professions[1:]:
        # prompt2 = p_template.format(profession2=profession2, jokes_output=resp1, profession1=profession1)
        # resp2 = llm_in_use(prompt2)
        llm_chain2 = LLMChain(prompt=p_template, llm=llm_in_use, verbose=verbose)
        resp2 = llm_chain2.predict(profession2=pff, jokes_output=resp1, profession1=profession1)
        print (f"======================={pff}=====================")
        print (resp2.strip())

**Using OpenAI**

In [7]:
print_jokes_multi_llmchain(openai_llm, template1, professions, p_template)

================== ORIGINAL jokes : lawyers ========================
1. How many lawyers does it take to change a lightbulb? None, they'd rather keep their clients in the dark. 
2. What do you call a lawyer gone bad? A corrupt-counsel.
3. Why did the lawyer cross the road? To get to the court house.
4. What do you call 5000 dead lawyers at the bottom of the ocean? A good start.
5. What's the difference between a lawyer and a vampire? A vampire only sucks your blood at night.
6. Why don't sharks attack lawyers? Professional courtesy.
7. How can you tell when a lawyer is lying? His lips are moving.
=======================truck drivers=====================
1. How many truck drivers does it take to change a lightbulb? One, they're always up for any challenge. 
2. What do you call a truck driver gone bad? A road-ragin' renegade.
3. Why did the truck driver cross the road? To deliver the goods.
4. What do you call 5000 dead truck drivers at the bottom of the ocean? An empty parking lot.
5. W

**Using Anthropic**

In [8]:
print_jokes_multi_llmchain(anthropic_llm, template1, professions, p_template)

================== ORIGINAL jokes : lawyers ========================
1. Why do lawyers wear neckties? To keep the foreskin from crawling up their faces.

2. How can you tell when a lawyer is lying? His lips are moving.

3. What's the difference between a jellyfish and a lawyer? One's a form of life, the other is a form of slime.

4. What's the difference between a lawyer and a vulture? The lawyer has better clothes.

5. Why won't sharks attack lawyers? Professional courtesy.

6. What do you call 25 lawyers skydiving from an airplane? Skeet.

7. What's the difference between a lawyer and a trampoline? You take your shoes off to jump on a trampoline.
=======================truck drivers=====================
1. Why do truck drivers wear neckties? To keep their double chins from jiggling.

2. How can you tell when a truck driver is lying? His lips are moving.

3. What's the difference between a jellyfish and a truck driver? One's a form of life, the other is a form of slime.

4. What's the

## Takeaways
- there are multiple ways to do the same thing
- LangChain is not technically needed to achieve any of this, these are simple demos and they could be done using only HTTP calls to the APIs
- the use-case of LangChain is really all about **expressing complex operations in terms of simple primitives**, i.e. it provides a nice set of abstractions to think of interacting with LLMs
- Anthropic jokes are often noticeably worse, often it is just a copy paste of a few words without any meaning-level change of the joke